The following notebook will get two different datasets of explanatory variables: temporal an non-temporal
related. 
In order to improve the speed time, this notebook will create the respective datasets and it will send a task to 
EarthEngine with a ReduceByRegion operation, we have proved that this method is faster than using the individual
calls to the API.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Literal

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import ee
from gee_scripts.get_sources import get_s1_collection, get_gldas, get_gpm, get_hansen, get_gpm_sum
from gee_scripts.get_sources import get_srtm, get_globcover, get_gedi, get_gldas_stats, get_extra_non_temporal
from gee_scripts import init_ee
from gee_scripts.parameters import best_kalimantan_phus, bad_stations

init_ee()

The objective would be to loop over the points or the dates...<br>
After testing this script https://code.earthengine.google.com/b18e876cca44266be704924b7354ddff <br>
I found out that the best way to do it is to loop over the dates, and then pass the reduceregions. <br>


# 1. Read data

In [ ]:
df = pd.read_csv('data/field_data_unique_coords_plus_bosf_and_spatial.csv', parse_dates=["date"])
assert df["date"].dtype == "datetime64[ns]"
print(len(df))
df.columns, df.source.unique()

In [ ]:
df = df[(df.phu_id.isin(best_kalimantan_phus)) & (~df.id.isin(bad_stations)) | (df.source == "bosf_NASA")]
print(len(df))

# # subset by unique id and date and show the lenght  

# Filter only by bosf
# df = df[df.source == "bosf_NASA"]


print(len(df.drop_duplicates(subset=["id", "date"])))

In [ ]:
# ##########33 THIS IS TEMPORARY #############

# For the second iteration I will only use the missing data from bosf

# missing_bosf = pd.read_csv("data/7_training_data/bosf/missing_points_dates.csv", parse_dates=["date"])

# # concatenate date and id to get unique id
# df["date_id"] = df["date"].dt.strftime("%Y-%m-%d") + "_" + df["id"].astype(str)

# # Filter in the missing bosf data
# df = df[df["date_id"].isin(missing_bosf["date_id"])]
# len(df)

# df

##########################
## Set type of output
##########################

In [ ]:
# This notebook can be run entirely, if we want to task the orders to GEE we'll set this variable to True
send_task = False

##########################

In [ ]:
unique_coords = df[["id", "lon", "lat"]].drop_duplicates()
unique_coords.head()
len(unique_coords)

In [ ]:
# Convert them as a geodataframe and save them
geometry = [Point(xy) for xy in zip(unique_coords.lon, unique_coords.lat)]
gdf = gpd.GeoDataFrame(unique_coords, geometry=geometry)

In [ ]:
# Read regions shapefile

shp_path = Path("data/0_shp/")
region_path = "regions_to_request_explanatory_all.gpkg"

gdf_regions = gpd.GeoDataFrame.from_file(shp_path/region_path)
gdf_regions

In [ ]:
# # Remove those date where the gwl measure is out of reasonable range
# upper_thres = 20
# lower_thres = -100

# df = df[(df.gwl_cm < upper_thres) & (df.gwl_cm > lower_thres)]

# # Get the coordinates of the individual points

# unique_coords = df[["id", 'lon', 'lat']].drop_duplicates()
# len(df), len(unique_coords)

In [ ]:
# Create geodataframe from x y coordinates
gdf_unique_coords = gpd.GeoDataFrame(unique_coords, geometry=gpd.points_from_xy(unique_coords.lon, unique_coords.lat), crs="EPSG:4326")

# Add the region id to each point
gdf_unique_coords = gpd.sjoin(gdf_unique_coords, gdf_regions[["region_id", "geometry"]], how="left", predicate="within")

In [ ]:
len(gdf_unique_coords)

# 2. Get temporal explanatory

In [ ]:
# Set selectors
from gee_scripts.parameters import base_selectors, s1_selectors, gldas_selectors, gpm_selectors, gpm_selectors_sum, hansen_selectors

In [ ]:
def get_temporal_explanatory(region_id, mode: Literal["strict", "extra"]="strict", position=1):
    """Get the explanatory temporal based variables.
    
    Parameters:
    region_id: int
        The region id to get the data for
    mode: Literal["strict", "extra"]
        strict: Only get the data for the exact date
        extra: Get the closest image to the date and adds the time difference as a variable
    position: int
        The position of the image selected in the collection, starts at 1
    """

    region = gdf_regions[gdf_regions.region_id == region_id].to_crs("EPSG:4326")[:]
    dates = df[df.id.isin(gdf_unique_coords[gdf_unique_coords.region_id == region_id].id.unique())].date.unique()
    points = gdf_unique_coords[gdf_unique_coords.region_id == region_id][["id", "geometry", "lat", "lon"]].to_crs("EPSG:4326")

    if mode == "extra":
        base_selectors + ["time_difference"]

    # print(len(dates), len(points))
    # Convert to ee elements

    ee_dates = ee.FeatureCollection(ee.List([ ee.Feature(None, {"date": date}) for date in dates]))
    ee_points = ee.FeatureCollection(points.__geo_interface__)
    ee_region = ee.FeatureCollection(region.__geo_interface__)

    print(f"Region {region_id} has {len(dates)} dates and {len(points)} points")

    def get_sources(date_feature):

        if mode=="strict":
            date_range = ee.Date(date_feature.get("date")).getRange('day')

            s1_composite = get_s1_collection(date_range, ee_region).median()

            return s1_composite.set({
            "numberOfBands" : s1_composite.bandNames().size(),
            "date" : ee.Date(date_feature.get("date"))
            })
        
        elif mode=="extra":
            requested_date = ee.Date(date_feature.get("date"))
            # Define a time window to search for images, e.g., +/- 30 days
            start_date = requested_date.advance(-30, 'day')
            end_date = requested_date.advance(30, 'day')

            # Get all S1 images in the time window
            s1_collection = get_s1_collection(ee.DateRange(start_date, end_date), ee_region)

            # Function to compute absolute difference in days between image date and requested date
            def compute_abs_difference(image):
                diff = ee.Number(image.date().difference(requested_date, 'day')).abs()
                return image.set('time_difference', diff)

            # Map over the collection to compute time difference
            s1_collection = s1_collection.map(compute_abs_difference)

            # Sort the collection by time difference
            sorted_collection = s1_collection.sort('time_difference')

            # Get the date of the position image in the collection

            selected_image = ee.Image(sorted_collection.toList(sorted_collection.size()).get(position-1))

            selected_image_date = selected_image.date()
            selected_image_time_difference = selected_image.get('time_difference')

            # Filter the collection to images that have the same date as the closest image
            images_same_date = s1_collection.filterDate(selected_image_date, selected_image_date.advance(1, 'day'))

            # Mosaic the images
            s1_image = images_same_date.median()

            return s1_image.set({
                "numberOfBands": s1_image.bandNames().size(),
                "date": requested_date,
                "time_difference": selected_image_time_difference
            })


    def reduce_composite(composite):

        # Filter the extra data with the matching date
        date = composite.get("date")
        date_range = ee.Date(date).getRange('day')
        time_difference = composite.get("time_difference")

        gldas_composite = get_gldas(date_range, ee_region)
        gpm_composite = get_gpm(date_range, ee_region)
        gpm_sum_composite = get_gpm_sum(date_range, ee_region)

        composite = (ee.Image(composite)
            .addBands(gldas_composite)
            .addBands(gpm_composite)
            .addBands(gpm_sum_composite)
        )

        return composite.reduceRegions(**{
         "collection" : ee_points,
         "reducer" : ee.Reducer.first(),
         "scale" : 10,
         "tileScale" : 16
        }).filter(ee.Filter.notNull(['VH'])).map(lambda feature: feature.set({
            "date" : date,
            "time_difference": time_difference,
        }))


    task = (ee_dates
         .map(get_sources)
         .filter(ee.Filter.gt('numberOfBands', 0))
         .map(reduce_composite).flatten()
    )

    # task_name = f"All_temporal_non_resample_at_all_region_{region_id}_dates_{len(dates)}_points_{len(points)}_with_date_lon_lat"
    task_name = f"all_temporal_extended_{region_id}_dates_{len(dates)}_points_{len(points)}brgm_pkeg"""

    if mode=="extra":
        task_name = task_name + f"_position_{position}"


    ee_task = ee.batch.Export.table.toDrive(**{
      "collection": task, 
      "folder" : "INDONESIA_GWL",
      "description": task_name,
      "selectors": base_selectors + s1_selectors + gldas_selectors + gpm_selectors + gpm_selectors_sum

    })

    # Uncoment to start the task
    not send_task or ee_task.start()
    print("Exported" if send_task else "Not exported", task_name)



#### SET PARAMETERS ####
mode:Literal["estrict", "extra"] = "strict"

if mode=="extra":
    base_selectors = list(set(base_selectors + ["time_difference"]))

# Region_id 20 is the bosf region, use mode="extra" to get the closest image to the date
[get_temporal_explanatory(region_id, mode=mode, position=3) for region_id in gdf_regions.region_id.unique() if region_id in [8]]


# 2.2 Get "yearly" temporal explanatory variables (Hansen)

In [ ]:
df.date.dt.year.unique()

In [ ]:
# get all the years from the field data
years = sorted([y for y in df.date.dt.year.unique() if y != 2013])
points = df[["id", "lon", "lat"]].drop_duplicates()

# This is only for the bosf region
points = gdf_unique_coords[gdf_unique_coords.region_id == 8][["id", "lon", "lat"]].drop_duplicates()
points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat), crs="EPSG:4326")
print(len(points))

ee_points = ee.FeatureCollection(points.__geo_interface__)

no_points = ee_points.size().getInfo()

print(f"Processing points {no_points}")

for year in years:
    
    # The dataset for 2020 is not available ,try with get_landsat_mosaic
    image = get_hansen(year)
    result = image.reduceRegions(**{
        "collection" : ee_points,
        "reducer" : ee.Reducer.first(),
        "scale" : 30,
        "tileScale" : 16
    }).map(lambda feature: feature.set("year", str(year)))



    task_name = f"Hansen_year_{year}_points_{no_points}_f_bosf"

    ee_task = ee.batch.Export.table.toDrive(**{
        "collection": result, 
        "folder" : "INDONESIA_GWL",
        "description": task_name,
        "selectors": base_selectors + hansen_selectors
    })
    not send_task or ee_task.start()
    print("Exported" if send_task else "Not exported", task_name)



## 3. Get non temporal explanatory variables (others)

In [ ]:
# This dataset is not too computational expensive, so we are not forced to chunk it
# We'll try to get all the points at once, not by region (so we won't filter by region)
region = gdf_regions.to_crs("EPSG:4326")[:]

# Below is just to use bosf region
region = gdf_regions[gdf_regions.region_id==8].to_crs("EPSG:4326")[:]


ee_region = ee.FeatureCollection(region.__geo_interface__)
points = gdf_unique_coords[["id", "geometry", "lat", "lon"]].rename(columns={"id": "id"}).to_crs("EPSG:4326")
ee_points = ee.FeatureCollection(points.__geo_interface__).filterBounds(ee_region)

print(ee_points.size().getInfo())   

In [ ]:
from gee_scripts.parameters import non_temporal_vars

# Remove time_difference from the base selectors
base_selectors = [selector for selector in base_selectors if selector != "time_difference"]

composite = (
    get_srtm()
        .addBands(get_globcover())
        .addBands(get_gedi(ee_region))
        .addBands(get_gldas_stats(ee_region))
        .addBands(get_extra_non_temporal())
)

result = composite.reduceRegions(**{
    "collection" : ee_points,
    "reducer" : ee.Reducer.first(),
    "scale" : 10,
    "tileScale" : 16
}).filter(ee.Filter.notNull(['canopy_height']))

no_points = ee_points.size().getInfo()
task_name = f"All_Non_temporal_points_{no_points}_bosf_and_all"


ee_task = ee.batch.Export.table.toDrive(**{
    "collection": result, 
    "folder" : "INDONESIA_GWL",
    "description":task_name,
    "selectors": base_selectors + non_temporal_vars
})


# Uncoment to start the task
not send_task or ee_task.start()

print("Exported" if send_task else "Not exported", task_name)

# 4. Merge explanatory variables

## 4.1 Read temporal variables

In [ ]:
from pathlib import Path

In [ ]:
# explanatory_path = Path("data/7_training_data/")
# dataset = "all"
# temporal_file_names_groups = {
#     "all" : [
#         "All_temporal_non_resample_at_all_region_1_dates_520_points_24_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_2_dates_1773_points_148_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_3_dates_479_points_1_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_4_dates_988_points_348_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_5_dates_1796_points_717_with_date.csv",
#         "All_temporal_non_resample_at_all_region_6_dates_489_points_43_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_7_dates_1274_points_477_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_8_dates_1671_points_220_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_9_dates_379_points_17_with_date_lon_lat.csv",
#         "All_temporal_non_resample_at_all_region_10_dates_846_points_77_with_date_lon_lat.csv",
#     ]
# }

# # Dataset is the name of the type of data we're using (high_corr or all) (it's assigned at the beginning of the notebook)
# temporal_file_names = temporal_file_names_groups[dataset]

# # get and concatenate all the dataframes
# temp_explanatory_dfs = pd.concat([
#             pd.read_csv(explanatory_path/file_name, parse_dates=["date"])
#             for file_name 
#             in temporal_file_names
#         ], 
# )

# temp_explanatory_dfs["date"] = pd.to_datetime(temp_explanatory_dfs["date"])
# temp_explanatory_dfs

In [ ]:
explanatory_path = Path("data/7_training_data/")
dataset = "brg_pkeg_time_diff"

# I modified the notebook on the 31/05/2024 to include the sum of the precipitation
temporal_precip_sum = {
    "all" : [
        "0_Precipitation_sum_non_resample_at_all_region_1_dates_520_points_24_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_2_dates_1773_points_148_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_3_dates_362_points_1_with_date_lon_lat.csv", # Using this there's only 362 where the other has 479
        "0_Precipitation_sum_non_resample_at_all_region_4_dates_988_points_348_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_5_dates_1796_points_718_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_6_dates_489_points_43_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_7_dates_1273_points_477_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_8_dates_1671_points_219_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_9_dates_379_points_17_with_date_lon_lat.csv",
        "0_Precipitation_sum_non_resample_at_all_region_10_dates_846_points_77_with_date_lon_lat.csv",
    ],
    "bosf" : [
        "bosf/all_temporal_extended_20_dates_644_points_381bosf_it_0.csv",
        "bosf/all_temporal_extended_20_dates_644_points_381bosf_it_1_filtered.csv",
    ],
    "brg_pkeg_time_diff" : [
        "bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg.csv",
        "bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_2_filtered.csv",
        "bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_3_filtered.csv",
        "bosf/all_temporal_extended_20_dates_644_points_381bosf_it_0.csv",
        "bosf/all_temporal_extended_20_dates_644_points_381bosf_it_1_filtered.csv",

    ]
}

# Dataset is the name of the type of data we're using (high_corr or all) (it's assigned at the beginning of the notebook)
temporal_file_names = temporal_precip_sum[dataset]

# get and concatenate all the dataframes
temp_explanatory_dfs = pd.concat([
            pd.read_csv(explanatory_path/file_name) 
            for file_name 
            in temporal_file_names
        ], 
)

temp_explanatory_dfs["date"] = pd.to_datetime(temp_explanatory_dfs["date"])
temp_explanatory_dfs

print(len(temp_explanatory_dfs))

temp_explanatory_dfs.columns

## 4.2 Read Hansen yearly variables

In [ ]:
hansen_file_names = [
    "data/7_training_data/Hansen_year_2018_points_2075_f.csv",
    "data/7_training_data/Hansen_year_2019_points_2075_f.csv",
    "data/7_training_data/Hansen_year_2020_points_2075_f.csv",
    "data/7_training_data/Hansen_year_2021_points_2075_f.csv",
    "data/7_training_data/Hansen_year_2022_points_2075_f.csv",
    "data/7_training_data/Hansen_year_2023_points_2075_f.csv",
    "data/7_training_data/bosf/Hansen_year_2016_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2017_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2018_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2019_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2020_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2021_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2022_points_377_f_bosf.csv",
    "data/7_training_data/bosf/Hansen_year_2023_points_377_f_bosf.csv",
]

hansen_df = pd.concat([
    pd.read_csv(file_name) 
    for file_name 
    in hansen_file_names
], axis=0)

hansen_df.columns

## 4.3 Read non temporal explanatory

In [ ]:
explanatory_path = Path("data/7_training_data/bosf/")

# As the non-temporal variables are the same for all the points, we just need to duplicate 
# their results into each of the dates of the points.
# i.e. 1 point with 10 dates will have the same non-temporal variables for each of the 10 dates.

non_temporal_file_name = "All_Non_temporal_points_408_bosf_and_all.csv"

# for bosf
# non_temporal_file_name = "All_Non_temporal_points_381_bosf.csv"

non_temporal_df = pd.read_csv(explanatory_path/non_temporal_file_name)
# drop lat and lon if they exist

columns_to_drop = ["lat", "lon", "date", "time_difference"]
non_temporal_df = non_temporal_df.drop(columns=[col for col in columns_to_drop if col in non_temporal_df.columns])
# non_temporal_df

print(non_temporal_df.columns)
print(len(non_temporal_df.id.unique()))

## 4.4 Create final explanatory variables dataset

In [ ]:
# Merge the non-temporal variables with the temporal ones
explanatory_df = temp_explanatory_dfs.merge(non_temporal_df, on="id")
len(explanatory_df)
explanatory_df.columns

In [ ]:
# Merge hansen data with year and id
explanatory_df["year"] = explanatory_df.date.dt.year
hansen_df["year"] = hansen_df["year"].astype(int)
explanatory_df = explanatory_df.merge(hansen_df[["id"] + hansen_selectors], on=["id", "year"], how="left")

# I get more values here because I have requested Hansen for all the years
explanatory_df

In [ ]:
from gee_scripts.parameters import explain_vars
export_vars = explain_vars + ['id', 'date', "time_difference"]
# Remove "doy" because we have not created it yet

export_vars.remove("doy")

# 4.5 Final step: Merge explanatory variables with response variable

In [ ]:
explanatory_with_response_var = df[["source", "id", "date", "gwl_cm", "lat", "lon"]].merge(
    explanatory_df[export_vars], on=["id", "date"]
)

# Add day of the year as a variable
explanatory_with_response_var["doy"] = explanatory_with_response_var.date.dt.dayofyear

print("lenght of base_df: ", len(explanatory_with_response_var))
# Drop duplicates on id and date
explanatory_with_response_var = explanatory_with_response_var.drop_duplicates(subset=["id", "date"])
print("lenght of explanatory_with_response_var after dropping duplicates: ", len(explanatory_with_response_var))

# Fill time_difference with 0 if is nan
explanatory_with_response_var["time_difference"] = explanatory_with_response_var["time_difference"].fillna(0)

explanatory_with_response_var.to_csv("data/7_training_data/bosf/explanatory_with_response_var_and_source_extra_bosf_brgm_all_extras.csv", index=False)
len(explanatory_with_response_var)

In [ ]:
explanatory_with_response_var.columns

In [ ]:
# # Export the data
explanatory_with_response_var.to_csv("data/7_training_data/bosf/explanatory_with_response_var_and_source_extra_sum_prec_brgm_exact_date.csv", index=False)

# 4.6 Optional (find missing points on second closest date)

In [ ]:

def get_missing_points(df, explanatory_with_response_var):
    """This is an extra step only for bosf.

        # We have gotten the data for the first closest date, this will leave some 
        # points without data (see notes.rst for more info), we need to get the data for the second closest date, but first we need to find the points that are missing data. 
        # We do that by just creating an unique id with the date and the id of the point and then we filter the points that are not in the explanatory_with_response_var dataframe.

        # Get the missing points"""

    df["date_id"] = df["date"].dt.strftime("%Y-%m-%d") + "_" + df["id"].astype(str)
    print(df.date_id.nunique())
    explanatory_with_response_var["date_id"] = explanatory_with_response_var["date"].dt.strftime("%Y-%m-%d") + "_" + explanatory_with_response_var["id"].astype(str)

    missing_points = df[
        (~df["date_id"].isin(explanatory_with_response_var["date_id"]))
    ]
    missing_points = missing_points.drop_duplicates(subset=["id", "date"])

    print("Missing points: ", len(missing_points))

    print(len(df)- len(missing_points) )

    # Export the missing points 
    return missing_points


missing_points = get_missing_points(df, explanatory_with_response_var)
missing_points.to_csv("data/7_training_data/bosf/missing_points_brgm.csv", index=False)

In [ ]:
# As I already requested all the data for the second closest date, I can just filter the points by
# The previous result.

# read second closest date data

all_temp_2nd_closest = pd.read_csv("data/7_training_data/bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_2.csv")

# Create the unique id with the date (YYYY-MM-DD) and the id of the point
all_temp_2nd_closest["date_id"] = all_temp_2nd_closest["date"].str.split("T").str[0] + "_" + all_temp_2nd_closest["id"].astype(str)

# Filter out the points that are not in the all_temp_2nd_closest dataframe
all_temp_missing_2nd = all_temp_2nd_closest[all_temp_2nd_closest["date_id"].isin(missing_points["date_id"])]

# Export the missing temporal data
print(len(all_temp_missing_2nd))

In [ ]:
all_temp_missing_2nd.to_csv("data/7_training_data/bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_2_filtered.csv", index=False)


In [ ]:
# Find the third closest date

all_temp_3rd_closest = pd.read_csv("data/7_training_data/bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_3.csv")

# Create the unique id with the date (YYYY-MM-DD) and the id of the point
all_temp_3rd_closest["date_id"] = all_temp_3rd_closest["date"].str.split("T").str[0] + "_" + all_temp_3rd_closest["id"].astype(str)

# Filter out the points that are not in the all_temp_3rd_closest dataframe and and not in the 2nd closest
all_temp_missing_3rd = all_temp_3rd_closest[all_temp_3rd_closest["date_id"].isin(missing_points["date_id"]) & (~all_temp_3rd_closest["date_id"].isin(all_temp_missing_2nd["date_id"]))]

all_temp_missing_3rd["time_difference"].describe()

all_temp_missing_3rd.to_csv("data/7_training_data/bosf/all_temporal_extended_8_dates_1559_points_31brgm_pkeg_position_3_filtered.csv", index=False)